In [4]:
'''
make_submission() generates predictions for the Kaggle Painter by Numbers competion
using simple features (image size, aspect ratio and bits/pixel^2)
author: Swaroop Krothapalli - extended code of small yello duck
https://github.com/swaroop7/painters
'''
import random
import time
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.cross_validation import KFold
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score  
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from scipy.stats import itemfreq
from sklearn import neighbors, linear_model
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import cross_validation
np.set_printoptions(precision=3, linewidth=100)

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

os.chdir('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features')

C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [1]:
input_file = open('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\train_conv5_1.txt')
output_file = open('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\train_processed_conv5_1.txt','w')

random = {}

output = set()


for line in input_file:
    line = line[2:]
    for each_word in line.split():
        words = each_word.split(":")
        output.add(words[0].strip())
    print len(output)

print 'part 1 done'

for x in output:
    random[x] = 0.


input_file.close()
input_file = open('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\train_conv5_1.txt')

for line in input_file:
    line = line[2:]
    for each_word in line.split():
        words = each_word.strip().split(":")
        random[words[0]] = round(float(words[1]), 4)
            
    for key,value in random.items():
        output_file.write(str(value)+' ')
        random[key] = 0.
    output_file.write('\n')
    

output_file.close()

27930
41529
52608
58964
64500
68575
70758
73342
75705
77261
78794
80340
81345
82469
83664
84221
84932
85442
86168
86781
87379
87756
88277
88817
89094
89264
89492
89690
90329
90544
91058
91423
91690
91796
91961
92280
92407
92648
92702
92935
93109
93169
93305
93384
93490
93721
93828
94072
part 1 done


In [2]:
input_file = open('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\test_conv5_1.txt')
output_file = open('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\test_processed_conv5_1.txt','w')

for line in input_file:
    line = line[2:]
    for each_word in line.split():
        words = each_word.strip().split(":")
        if words[0] in output:
            random[words[0]] = round(float(words[1]), 4)
            
    for key,value in random.items():
        output_file.write(str(value)+' ')
        random[key] = 0.
    print 'line done'
    output_file.write('\n')
    
input_file.close()
output_file.close()

line done
line done
line done
line done
line done
line done
line done
line done
line done
line done


In [6]:
'''
x_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april9\\train_processed_conv5_1_APRIL8.txt')
y_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\y_train_conv5_1.txt')
x_test = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april9\\test_conv5_1_processed_APRIL8.txt')
y_test = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\y_test_conv5_1.txt')
'''

x_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\train_processed_conv5_1.txt')
y_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\y_train_conv5_1.txt')
x_test = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\test_processed_conv5_1.txt')
y_test = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\y_test_conv5_1.txt')


print x_train.shape

'''
rand_numbers = []

for x in range(1,500):
    rand_numbers.append(random.randint(1,79928))
    
new_x_train = x_train[:, rand_numbers]

new_x_test = x_test[:, rand_numbers]
'''
print x_train.shape
print x_test.shape
print y_train.shape
print y_test.shape

(48L, 94072L)
(48L, 94072L)
(10L, 94072L)
(48L,)
(10L,)


In [7]:
def print_results(clf, y_test, y_pred, y_pred_prob):
    #y_pred_prob = clf.predict_proba(y_test)[:,1]
    #y_pred = clf.predict(y_test)
    print 'ROC - ',roc_auc_score(y_test, y_pred_prob)
    print 'Accuracy - ',accuracy_score(y_test, y_pred)
    print 'Confusion Matrix - ', confusion_matrix(y_test, y_pred)
    #print 'Precision - ',precision_score(y_test, y_pred ),'Recall - ',recall_score(y_test, y_pred),'F1- Score',f1_score(y_test, y_pred),'\n'
    target_names = ['class 0', 'class 1']
    print classification_report(y_test, y_pred, target_names=target_names)

In [8]:
print x_train.shape, y_train.shape
#training, testing, y_training, y_testing = train_test_split(new_x_train, y_train, test_size=0.2, stratify=y_train, random_state=42)
actual_training, validation, y_actual_training, y_validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=42)
#print training.shape, testing.shape
#print y_training, y_testing
#print itemfreq(y_training)
#print itemfreq(y_testing)
print itemfreq(y_validation)
#print itemfreq(y_train)

print y_validation
# print y_actual_training
# print actual_training


(48L, 94072L) (48L,)
[[ 0.  6.]
 [ 1.  4.]]
[ 0.  0.  1.  0.  0.  1.  1.  1.  0.  0.]


In [9]:
#Random Forest
print "ACCURACY RESUTLS"    
print "Mean Accuracy of 5 fold CV","\t", "STD","\t", "Validation ","\t", "Test"        

for i in xrange(1):
    clf = RandomForestClassifier(n_estimators=42, class_weight='balanced',  random_state = 42)
    clf.fit(actual_training, y_actual_training)
    scores = cross_validation.cross_val_score(clf, actual_training, y_actual_training, cv=5)
    y_pred_valid = clf.predict(validation)
    print i, scores.mean(), scores.std(), accuracy_score(y_validation, y_pred_valid), accuracy_score(y_test, clf.predict(x_test))
    
print_results(clf, y_test, clf.predict(x_test), clf.predict_proba(x_test)[:,1] )

ACCURACY RESUTLS
Mean Accuracy of 5 fold CV 	STD 	Validation  	Test
0 0.737698412698 0.0770332031432 0.7 0.6
ROC -  0.625
Accuracy -  0.6
Confusion Matrix -  [[4 2]
 [2 2]]
             precision    recall  f1-score   support

    class 0       0.67      0.67      0.67         6
    class 1       0.50      0.50      0.50         4

avg / total       0.60      0.60      0.60        10



In [10]:
#XGB
    
learning_r_col = [0.0001, 0.001, 0.01, 0.02, 0.03, 0.04,0.05,0.06,0.07,0.08,0.09,0.1]    
    
print "ACCURACY RESUTLS"    
print "Mean Accuracy of 5 fold CV","\t", "STD","\t", "Validation ","\t", "Test"        

    
for i in xrange(1):
    clf = xgb.XGBClassifier(max_depth=2, n_estimators=5, learning_rate=0.001) #objective='multi:softprob'
    clf.fit(actual_training, y_actual_training)
    scores = cross_validation.cross_val_score(clf, actual_training, y_actual_training, cv=5)
    y_pred_valid = clf.predict(validation)
    print i, "\t",scores.mean(),"\t", scores.std(), "\t",accuracy_score(y_validation, y_pred_valid), "\t",accuracy_score(y_test, clf.predict(x_test))

print_results(clf, y_test, clf.predict(x_test), clf.predict_proba(x_test)[:,1] )

ACCURACY RESUTLS
Mean Accuracy of 5 fold CV 	STD 	Validation  	Test
0 	0.610317460317 	0.168814579666 	0.6 	0.4
ROC -  0.458333333333
Accuracy -  0.4
Confusion Matrix -  [[2 4]
 [2 2]]
             precision    recall  f1-score   support

    class 0       0.50      0.33      0.40         6
    class 1       0.33      0.50      0.40         4

avg / total       0.43      0.40      0.40        10



In [11]:
#Logistic
    
print "ACCURACY RESUTLS"    
print "Mean Accuracy of 5 fold CV","\t", "STD","\t", "Validation ","\t", "Test"    
    
for i in xrange(1):  # , max_iter = i
    clf = linear_model.LogisticRegression( class_weight='balanced', penalty = 'l1')
    clf.fit(actual_training, y_actual_training)
    scores = cross_validation.cross_val_score(clf, actual_training, y_actual_training, cv=5)
    y_pred_valid = clf.predict(validation)
    print i, scores.mean(), scores.std(), accuracy_score(y_validation, y_pred_valid), accuracy_score(y_test, clf.predict(x_test))
    
print_results(clf, y_test, clf.predict(x_test), clf.predict_proba(x_test)[:,1] )

ACCURACY RESUTLS
Mean Accuracy of 5 fold CV 	STD 	Validation  	Test
0 0.709126984127 0.102651298042 0.7 0.5
ROC -  0.458333333333
Accuracy -  0.5
Confusion Matrix -  [[3 3]
 [2 2]]
             precision    recall  f1-score   support

    class 0       0.60      0.50      0.55         6
    class 1       0.40      0.50      0.44         4

avg / total       0.52      0.50      0.51        10



In [19]:
#Linear SVC
    
print "ACCURACY RESUTLS of 5 fold CV"    
print "Mean Accuracy ","\t", "STD","\t", "Validation ","\t", "Test"    

iter = [5,10,100,500,1000]

for i in xrange(1):
    clf = LinearSVC(class_weight = 'balanced', dual = False)
    clf.fit(actual_training, y_actual_training)
    scores = cross_validation.cross_val_score(clf, actual_training, y_actual_training, cv=5 )
    y_pred_valid = clf.predict(validation)
    print i, "\t",scores.mean(),"\t", scores.std(), "\t",accuracy_score(y_validation, y_pred_valid), "\t",accuracy_score(y_test, clf.predict(x_test))

target_names = ['class 0', 'class 1']
print 'Confusion Matrix - ', confusion_matrix(y_test, clf.predict(x_test))
print classification_report(y_test, clf.predict(x_test), target_names=target_names)

ACCURACY RESUTLS of 5 fold CV
Mean Accuracy  	STD 	Validation  	Test
0 	0.822619047619 	0.0782822962989 	0.9 	0.5
Confusion Matrix -  [[3 3]
 [2 2]]
             precision    recall  f1-score   support

    class 0       0.60      0.50      0.55         6
    class 1       0.40      0.50      0.44         4

avg / total       0.52      0.50      0.51        10



## Visualize the 5-1 Layer

In [14]:
import numpy as np
import PIL
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
import time
#from IPython.display import Image
from IPython.display import display
%matplotlib inline
import os

os.chdir('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\5_1')

list_im = []

for i in xrange(1,507):
    k = i*15
    arr = np.genfromtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\das2_conv5_1.txt', skip_header=k, max_rows = 14)
    
    arr = arr/np.amax(arr)

    im = Image.fromarray(np.uint8(cm.gist_heat(arr)*255))
    
    #imshow(im)
    im.save('output'+str(i)+'.jpg')
    list_im.append('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\april27\\5_1\\output'+str(i)+'.jpg') 
    


imgs    = [ PIL.Image.open(i) for i in list_im ]
# pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )

# save that beautiful picture
imgs_comb = PIL.Image.fromarray( imgs_comb)
imgs_comb.save( 'Trifecta.jpg' )    

# for a vertical stacking it is simple: use vstack
imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
imgs_comb = PIL.Image.fromarray( imgs_comb)
imgs_comb.save( 'Trifecta_vertical.jpg' )

C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: RuntimeWarning: invalid value encountered in divide
C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\matplotlib\colors.py:494: RuntimeWarning: invalid value encountered in less
  cbook._putmask(xa, xa < 0.0, -1)
